In [3]:
import numpy as np

with open("../data/day18test.txt", "r") as f:
    input = f.read()

data = input.split("\n")
data

['[1,2]',
 '[[1,2],3]',
 '[9,[8,7]]',
 '[[1,9],[8,5]]',
 '[[[[1,2],[3,4]],[[5,6],[7,8]]],9]',
 '[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]',
 '[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]']

In [2]:
# Puzzle 1